# 一些关于CNN（卷积神经网络）的基本知识

### 概述

深度学习算法在计算机视觉中有重要的应用  
CNN是其中非常重要的一种：图像检测、分类和检索（识图）、超分辨率重构、人脸识别  
核心在于**特征提取**  
卷积神经网络与传统神经网络的区别： **高维化** 3D数据 h w c  
整体架构：**输入层（数据集）+卷积层（特征提取）+池化层（压缩）+全连接层（输入层→隐层）**

### *卷积层  
做每张图片中不同的区域都有不同的重要性（e.g 主体、背景），因此要先对其进行区域划分，每个区域有不同的像素点  
如何描述该区域的特征→计算**特征值**  
特征值的得到通过一组权重参数→每个像素点按照不同的权重参数加权求和 即得到每个小区域的特征值 （按照求内积的规则）（有时还要加**常数bias修正**）
权重参数和区域划分的选取：**卷积核** 是一种全局的filter （**卷积参数共享** why？减少参数个数，防止过拟合）

为什么会有3D？  
图像的**颜色通道**，即 c dimension  
每个颜色通道要分别用相同的卷积核做计算，并对结果求和  
而每次卷积完后输出结果的c是由卷积核的选取个数决定的

不同的卷积核将给出不同的**特征图**→特征更加丰富  
一共做多少次过滤才足够？对应着提取出不同层次的信息  
而每一次过滤都在上一次得到的3D特征图的基础上，而不是在原始数据上  
总体的参数包括：卷积核尺寸、滑动窗口步长、边缘填充、卷积核个数  
不同的卷积核大小和滑动窗口步长将得到不同的特征图大小 步长越小图越大、越细节  
一般来说对图像任务：步长1；文本识别任务：步长可以更大etc

**边缘填充**：注意一个问题，边界上的点因为卷积核移动的限制，对特征值的贡献次数将更少  
but 边界点不一定就是图像中不重要的点  
那，怎么办？  
和我们在沙堆中做的类似的 手动加上一圈数值为0的边界 （+pad 1）

### *池化层
因为卷积完后得到的矩阵非常大，所以要通过池化层进一步**压缩特征**\
池化算法有很多，几个栗子：\
**max pooling**:选出每个区域中最大的数，即提取最重要的特征 这是目前最普遍也最准确的做法\
**average pooling**

### *感受野
最后一步中一个特征值是由最初多少个格点参与计算得到的  
感受野越大越好：一个特征值所囊括的特征越多  
e.g why three 3乘3 instead of 7乘7 ? 更细致、更少参数、更多次线性操作


### *整个网络的神经任务的进行
  
<img src="./fig/cnn (2).png" width="40%"/>

每一次卷积之后都要加一层**非线性变换RELU**  
在这些组合之间间隔地加入池化压缩层  
最后的**全连接层FC**能给出图像对不同类别的符合程度（训练结果）  
但FC只能处理一维数据，所以将卷积、池化后的3D矩阵拉成向量  

层数计算：只有带参数计算的为一层 卷积层&FC层  

#### 几种经典网络架构
**Alexnet经典网络（2012）**    
8层网络、5层卷积+3层全连接  
问题在于卷积核太大（特征很粗）、没有加pad  
<img src="./fig/alexnet.png" width="40%">


**VGG网络（2014）**  
不同的版本 最主流的版本为红框内的  
3*3卷积核 更细腻 网络层数更多  
每次pooling之后，为了弥补损失的信息→特征图翻倍
但训练时间更长（several days

那是不是层数越多效果越好？the depth of learning？ That's not the case.  
在之前的特征图基础上再提取不一定会有更好的果效  
<img src="./fig/vgg.png" width="40%">


**残差网络Resnet**  
筛选出多余的层中效果好的  
同等映射：通过改变不同效果的层的权重来决定其是否取多大的作用  
比较中间那部是否存在的效果来决定其权重  
主要用于特征提取（网络有分类和回归的不同功）  

<img src="./fig/resnet.png" width="40%">




# 下面是CNN的编程实践

第一步，导入各个需要的库
torch装了两百年...

In [1]:
import torch                    #torch是一个张量处理库
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms    #torchvision库中有相关的网络结构
%matplotlib inline         
import matplotlib.pyplot as plt
import numpy as np

首先读取数据
- 构建训练集和测试集
- DataLoader来迭代取数据

In [2]:
#定义超参数
input_size=28 #图像是28*28*1
num_classes=10 #标签的种类数（指把图像分成多少类）
num_epochs=3 #训练总循环周期
batch_size=64 #一个批次64张图片

#这里使用的都是python模块的内置数据集
#训练集导入
train_dataset=datasets.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)

#测试集导入
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

#构建batch数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

23.8%


KeyboardInterrupt: 

卷积网络模块构建
- 一般卷积层，relu层，池化层可以写成一个套餐
- 注意卷积最后结果还是一个特征图，需要把图转换成向量才能做分类或者回归任务

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #第一个卷积套餐
        self.conv1 = nn.Sequential(         # 输入大小 (1, 28, 28)
            nn.Conv2d(
                in_channels=1,              # 灰度图，如果RGB图就是3
                out_channels=16,            # 要得到几多少个特征图
                kernel_size=5,              # 卷积核大小 5*5
                stride=1,                   # 步长
                padding=2,                  # 即+pad几 如果希望卷积后大小跟原来一样，需要设置padding=(kernel_size-1)/2 if stride=1
            ),                              # 输出的特征图为 (16, 28, 28)
            nn.ReLU(),                      # relu层
            nn.MaxPool2d(kernel_size=2),    # 进行池化操作（2x2 区域）, 输出结果为： (16, 14, 14)
        )
        self.conv2 = nn.Sequential(         # 下一个套餐的输入 (16, 14, 14)
            nn.Conv2d(16, 32, 5, 1, 2),     # 按照conv1中的参数顺序设置，输出 (32, 14, 14)
            nn.ReLU(),                      # relu层
            nn.MaxPool2d(2),                # 输出 (32, 7, 7)
        )
        
        #最后一步是将三维数据拉成向量，再连上全连接层
        #第一个参数为向量长度，第二个参数为图像种类数
        self.out = nn.Linear(32 * 7 * 7, 10)   # 全连接层得到的结果

   #连接网络，进行运算
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)           # flatten操作，结果为：(batch_size, 32 * 7 * 7)
        output = self.out(x)
        return output

定义评估标准：准确率

In [ ]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1] 
    rights = pred.eq(labels.data.view_as(pred)).sum() 
    return rights, len(labels) 

接着就可以开始训练模型了:)

In [ ]:
%%time
# 实例化
net = CNN() 
#损失函数
criterion = nn.CrossEntropyLoss() 
#优化器
optimizer = optim.Adam(net.parameters(), lr=0.001) #定义优化器，普通的随机梯度下降算法

#开始训练循环
for epoch in range(num_epochs):
    #当前epoch的结果保存下来
    train_rights = [] 
    
    for batch_idx, (data, target) in enumerate(train_loader):  #针对容器中的每一个批进行循环
        net.train()                             
        output = net(data) 
        loss = criterion(output, target) 
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 
        right = accuracy(output, target) 
        train_rights.append(right) 

    
        if batch_idx % 100 == 0: #每隔100次计算一下准确率
            
            net.eval() 
            val_rights = [] 
            
            for (data, target) in test_loader:
                output = net(data) 
                right = accuracy(output, target) 
                val_rights.append(right)
                
            #准确率计算
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))

            print('当前epoch: {} [{}/{} ({:.0f}%)]\t损失: {:.6f}\t训练集准确率: {:.2f}%\t测试集正确率: {:.2f}%'.format(
                epoch, batch_idx * batch_size, len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss.data, 
                100. * train_r[0].numpy() / train_r[1], 
                100. * val_r[0].numpy() / val_r[1]))